In [4]:
import os
import pandas as pd
import numpy as np
import datetime

# we are going to go quarter by quarter and see if there are any large airport routes from major carriers that dissapear from quarter to quarter
# this can be signified by the top carrier in a route disappearing and the amount of people using that route year over year dropping
# let's look into more possibilities if we have time

path_to_routes = "/Users/tristanbrigham/Desktop/Citadel Datathon/Local Important Data/Consumer_Airfare_Report__Table_6_-_Contiguous_State_City-Pair_Markets_That_Average_At_Least_10_Passengers_Per_Day.csv"

routes_df = pd.read_csv(path_to_routes)

custom_id = "citadel_id"


max_passenger_change = 0.15      # the largest percentage change in passengers that we will allow without flagging per quarter
ms_del = 0.25                    # the largest percentage change in ms that we will tolerate without flagging


# This function is used to create the custom codes that we use
# to index the airport pairing mappings
def conv_str(num1, num2):

    s = sorted([num1, num2])
    return str(s[0]) + str(s[1])


# apply the code generation function
routes_df[custom_id] = routes_df.apply(lambda row: conv_str(row['citymarketid_1'], row['citymarketid_2']), axis=1)
routes_df["dropped_q_over_q"] = False
routes_df["pass_and_lg_carrier_chng"] = False
routes_df["fare_and_lg_carrier_chng"] = False
routes_df["ms_change"] = False

routes_df["pass_chng_pct"] = 0
routes_df["fare_chng_pct"] = 0
routes_df["lg_fare_chng_pct"] = 0
routes_df["lf_fare_chng_pct"] = 0
routes_df["lg_carr_chng"] = 0
routes_df["lf_carr_chng"] = 0
routes_df["lg_ms_chng"] = 0
routes_df["lf_ms_chng"] = 0
routes_df["amt_quarters_past"] = 0


cols_frame = routes_df.columns
S_TEST = True
total_cancelled = pd.DataFrame(columns = cols_frame)

# the first, last dataset
last_q = 1996
last_y = 4
last_q_mappings = routes_df[(routes_df["Year"] == last_q) & (routes_df["quarter"] == last_y)]



/var/folders/d2/lsvbp7p92gnc5ql59sc72rx40000gn/T/ipykernel_29295/118714035.py:12: DtypeWarning: Columns (16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  routes_df = pd.read_csv(path_to_routes)


In [5]:
cols_frame

Index(['Year', 'quarter', 'citymarketid_1', 'citymarketid_2', 'city1', 'city2',
       'nsmiles', 'passengers', 'fare', 'carrier_lg', 'large_ms', 'fare_lg',
       'carrier_low', 'lf_ms', 'fare_low', 'table_1_flag', 'Geocoded_City1',
       'Geocoded_City2', 'tbl6pk', 'state_1', 'state_2', 'citadel_id',
       'dropped_q_over_q', 'pass_and_lg_carrier_chng',
       'fare_and_lg_carrier_chng', 'ms_change', 'pass_chng_pct',
       'fare_chng_pct', 'lg_fare_chng_pct', 'lf_fare_chng_pct', 'lg_carr_chng',
       'lf_carr_chng', 'lg_ms_chng', 'lf_ms_chng', 'amt_quarters_past'],
      dtype='object')

In [6]:
def check_exist(year, quarter, c1, c2):

    return pd.concat([routes_df[(routes_df["Year"] == year) & (routes_df["quarter"] == quarter) & (routes_df["citymarketid_1"] == c1) & (routes_df["citymarketid_2"] == c2)],  routes_df[(routes_df["Year"] == year) & (routes_df["quarter"] == quarter) & (routes_df["citymarketid_1"] == c2) & (routes_df["citymarketid_2"] == c1)]])

def check_exist_idx(year, quarter, idx):

    return routes_df[(routes_df["Year"] == year) & (routes_df["quarter"] == quarter) & (routes_df[custom_id] == idx)]

def get_quarters_past(row, i_year, i_quarter):

    t_year = i_year
    t_quar = i_quarter
    exists = True
    amt_back = 0

    while exists:

        amt_back += 1
        t_quar -= 1

        if t_quar == 0:

            t_year -= 1
            t_quar = 4

        if t_year == 1996:

            return amt_back


        exists = not check_exist_idx(t_year, t_quar, row[custom_id]).empty
    
    return amt_back


print(check_exist_idx(2000, 2, 3085234699).empty)
print(check_exist(2000, 2, 34699, 30852))

True
Empty DataFrame
Columns: [Year, quarter, citymarketid_1, citymarketid_2, city1, city2, nsmiles, passengers, fare, carrier_lg, large_ms, fare_lg, carrier_low, lf_ms, fare_low, table_1_flag, Geocoded_City1, Geocoded_City2, tbl6pk, state_1, state_2, citadel_id, dropped_q_over_q, pass_and_lg_carrier_chng, fare_and_lg_carrier_chng, ms_change, pass_chng_pct, fare_chng_pct, lg_fare_chng_pct, lf_fare_chng_pct, lg_carr_chng, lf_carr_chng, lg_ms_chng, lf_ms_chng, amt_quarters_past]
Index: []

[0 rows x 35 columns]


In [7]:

tot_len = len(routes_df)

# go through each year
for year in range(1997, 2020):

    print(f"YEAR: {year}")

    for quarter in range(1, 5):

        print(f"QUARTER: {quarter}")

        curr_frame = routes_df[(routes_df["Year"] == year) & (routes_df["quarter"] == quarter)]


        ## CHECK 1
        # check 1: check if any disappeared quarter over quarter last_q_mappings
        dropped_routes = last_q_mappings[~last_q_mappings[custom_id].isin(curr_frame[custom_id])]
        dropped_routes["dropped_q_over_q"] = True
        dropped_routes["amt_quarters_past"] = dropped_routes.apply(lambda row: get_quarters_past(row, last_y, last_q), axis=1)

        total_cancelled = pd.concat([total_cancelled, dropped_routes])

        # # CHECK 2:
        # check 2: check if variables below (largest carrier change, amt people change, mkt share change) mean major player left
        # going to start with decrease in passengers
        # we are going to look whether the carrier that is the largest and the carrier that is the smallest ended up changing

        for idx, row in curr_frame.iterrows():

            if idx % 10000 == 0:

                print(f"IDX: {idx} / {tot_len} @ {datetime.datetime.now()}")


            # trying incase that flight didn't exist the year before
            try:
    
                pass_pct_chng = row["passengers"] / routes_df[(routes_df["Year"] == last_q) & (routes_df["quarter"] == last_y) & (routes_df[custom_id] == row[custom_id])]["passengers"].values[0]
                fare_pct_chng = row["fare"] / routes_df[(routes_df["Year"] == last_q) & (routes_df["quarter"] == last_y) & (routes_df[custom_id] == row[custom_id])]["fare"].values[0]
                fare_lg_pct_chng = row["fare_lg"] / routes_df[(routes_df["Year"] == last_q) & (routes_df["quarter"] == last_y) & (routes_df[custom_id] == row[custom_id])]["fare_lg"].values[0]
                fare_low_pct_chng = row["fare_low"] / routes_df[(routes_df["Year"] == last_q) & (routes_df["quarter"] == last_y) & (routes_df[custom_id] == row[custom_id])]["fare_low"].values[0]
                
                F_lg_carr_chng = row["carrier_lg"] != routes_df.loc[(routes_df["Year"] == last_q) & (routes_df["quarter"] == last_y) & (routes_df[custom_id] == row[custom_id])]["carrier_lg"].values[0]
                F_lf_carr_chng = row["carrier_low"] != routes_df.loc[(routes_df["Year"] == last_q) & (routes_df["quarter"] == last_y) & (routes_df[custom_id] == row[custom_id])]["carrier_low"].values[0]
                F_lg_ms_chng = (row["large_ms"] - routes_df[(routes_df["Year"] == last_q) & (routes_df["quarter"] == last_y) & (routes_df[custom_id] == row[custom_id])]["large_ms"].values[0]) 
                F_lf_ms_chng = (row["lf_ms"] - routes_df[(routes_df["Year"] == last_q) & (routes_df["quarter"] == last_y) & (routes_df[custom_id] == row[custom_id])]["lf_ms"].values[0]) 


                # change percentages
                row_dict = {
                    'Year' : row['Year'],
                    'quarter' : row['quarter'],
                    'citymarketid_1' : row['citymarketid_1'],
                    'citymarketid_2' : row['citymarketid_2'],
                    'city1' : row['city1'],
                    'city2' : row['city2'],
                    'nsmiles' : row['nsmiles'],
                    'passengers' : row['passengers'],
                    'fare' : row['fare'],
                    'carrier_lg' : row['carrier_lg'],
                    'large_ms' : row['large_ms'],
                    'fare_lg' : row['fare_lg'],
                    'carrier_low' : row['carrier_low'],
                    'lf_ms' : row['lf_ms'],
                    'fare_low' : row['fare_low'],
                    'table_1_flag' : row['table_1_flag'],
                    'Geocoded_City1' : row['Geocoded_City1'],
                    'Geocoded_City2' : row['Geocoded_City2'],
                    'tbl6pk' : row['tbl6pk'],
                    'state_1' : row['state_1'],
                    'state_2' : row['state_2'],
                    'citadel_id' : row['citadel_id'],
                    'dropped_q_over_q' : False,
                    'pass_and_lg_carrier_chng' : False,
                    'fare_and_lg_carrier_chng' : False,
                    'ms_change' : False,
                    'pass_chng_pct' : pass_pct_chng,
                    'fare_chng_pct' : fare_pct_chng,
                    'lg_fare_chng_pct' : fare_lg_pct_chng,
                    'lf_fare_chng_pct' : fare_low_pct_chng,
                    'lg_carr_chng' : F_lg_carr_chng,
                    'lf_carr_chng' : F_lf_carr_chng,
                    'lg_ms_chng' : F_lg_ms_chng,
                    'lf_ms_chng' : F_lf_ms_chng,
                    'amt_quarters_past' : 1,
                }


                # create flags
                F_pass_pct_chng = pass_pct_chng - 1 < max_passenger_change
                F_fare_pct_chng = fare_pct_chng > max_passenger_change
                F_lg_ms_chng = (F_lg_ms_chng - 1) > ms_del
                F_lf_ms_chng = (F_lf_ms_chng - 1) > ms_del

                # if the passenger percentage dropped by too much or fares went up by too much
                # and the top carrier has to change (!)

                if (F_pass_pct_chng and (F_lg_carr_chng or F_lf_carr_chng)):
                    row_dict["pass_and_lg_carrier_chng"] = True
                
                if (F_fare_pct_chng and (F_lg_carr_chng or F_lf_carr_chng)):
                    row_dict["fare_and_lg_carrier_chng"] = True

                if F_lg_ms_chng:
                    row_dict["ms_change"] = True
                    
                if F_lf_ms_chng:
                    row_dict["ms_change"] = True
                    
                
                if (F_pass_pct_chng and (F_lg_carr_chng or F_lf_carr_chng)) | (F_fare_pct_chng and (F_lg_carr_chng or F_lf_carr_chng)) | F_lg_ms_chng | F_lf_ms_chng:
                    row_dict["amt_quarters_past"] = get_quarters_past(row, last_y, last_q)
                    total_cancelled = pd.concat([total_cancelled, pd.DataFrame(row_dict)], ignore_index = True)
                

                # print(total_cancelled)

            
            except:

                pass


        # total_cancelled_no_1996 = total_cancelled[~total_cancelled["Year"] == 1996]
        total_cancelled.to_csv(f"~/Desktop/cancelled_w_more_stats/{year}_{quarter}_basic_cancelled.csv", index=False)

        # update the dataframe from last quarter
        last_q_mappings = curr_frame
        last_q = quarter
        last_y = year




YEAR: 1997
QUARTER: 1


/var/folders/d2/lsvbp7p92gnc5ql59sc72rx40000gn/T/ipykernel_29295/731333169.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dropped_routes["dropped_q_over_q"] = True
/var/folders/d2/lsvbp7p92gnc5ql59sc72rx40000gn/T/ipykernel_29295/731333169.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dropped_routes["amt_quarters_past"] = dropped_routes.apply(lambda row: get_quarters_past(row, last_y, last_q), axis=1)


IDX: 20000 / 587610 @ 2023-07-29 02:47:09.594018
QUARTER: 2
IDX: 30000 / 587610 @ 2023-07-29 03:06:48.350536
QUARTER: 3
QUARTER: 4
IDX: 40000 / 587610 @ 2023-07-29 03:11:58.547083
YEAR: 1998
QUARTER: 1
QUARTER: 2
IDX: 50000 / 587610 @ 2023-07-29 03:17:28.682855
QUARTER: 3
QUARTER: 4
IDX: 60000 / 587610 @ 2023-07-29 03:22:56.912982
YEAR: 1999
QUARTER: 1
QUARTER: 2
IDX: 70000 / 587610 @ 2023-07-29 03:28:43.148498
QUARTER: 3
QUARTER: 4
IDX: 80000 / 587610 @ 2023-07-29 03:34:17.472087
YEAR: 2000
QUARTER: 1
QUARTER: 2
IDX: 90000 / 587610 @ 2023-07-29 03:40:07.801901
QUARTER: 3
IDX: 100000 / 587610 @ 2023-07-29 03:45:16.325716
QUARTER: 4
YEAR: 2001
QUARTER: 1
IDX: 110000 / 587610 @ 2023-07-29 03:51:33.363168
QUARTER: 2
QUARTER: 3
IDX: 120000 / 587610 @ 2023-07-29 03:57:38.735162
QUARTER: 4
YEAR: 2002
QUARTER: 1
IDX: 130000 / 587610 @ 2023-07-29 04:04:38.451650
QUARTER: 2
QUARTER: 3
IDX: 140000 / 587610 @ 2023-07-29 04:10:07.890510
QUARTER: 4
YEAR: 2003
QUARTER: 1
IDX: 150000 / 587610 @ 2023-

In [8]:

# total_cancelled_no_1996 = total_cancelled[~total_cancelled["Year"] == 1996]

total_cancelled.to_csv("~/Desktop/cancelled_w_more_stats/stats_basic_cancelled.csv", index=False)
total_cancelled.to_csv("~/Desktop/stats_basic_cancelled.csv", index=False)

In [9]:
total_cancelled

,Year,quarter,citymarketid_1,citymarketid_2,city1,city2,nsmiles,passengers,fare,carrier_lg,...,ms_change,pass_chng_pct,fare_chng_pct,lg_fare_chng_pct,lf_fare_chng_pct,lg_carr_chng,lf_carr_chng,lg_ms_chng,lf_ms_chng,amt_quarters_past
14731,1996,4,30721,34488,"Boston, MA (Metropolitan Area)","Reading, PA",293,13,243.78,US,...,False,0,0,0,0,0,0,0,0,1
14747,1996,4,33198,30928,"Kansas City, MO","Wichita, KS",186,30,60.77,NJ,...,False,0,0,0,0,0,0,0,0,1
14751,1996,4,30599,31953,"Birmingham, AL","Gainesville, FL",375,10,159.43,DL,...,False,0,0,0,0,0,0,0,0,1
14763,1996,4,31252,33495,"Daytona Beach, FL","New Orleans, LA",556,13,125.37,DL,...,False,0,0,0,0,0,0,0,0,1
14766,1996,4,30627,33198,"Bismarck/Mandan, ND","Kansas City, MO",600,11,234.54,NW,...,False,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
520338,2019,3,35323,33192,"Bristol/Johnson City/Kingsport, TN","Sacramento, CA",2140,10,377.92,DL,...,False,0,0,0,0,0,0,0,0,2
520344,2019,3,35323,34057,"Bristol/Johnson City/Kingsport, TN","Portland, OR",2167,10,378.37,DL,...,False,0,0,0,0,0,0,0,0,2
520350,2019,3,35380,34027,"Traverse City, MI","West Palm Beach/Palm Beach, FL",1282,11,336.12,DL,...,False,0,0,0,0,0,0,0,0,1
520356,2019,3,35412,30928,"Knoxville, TN","Wichita, KS",756,11,296.71,AA,...,False,0,0,0,0,0,0,0,0,2
